In [77]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import json
import random

In [78]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


TensorFlow version: 2.17.1
Num GPUs Available:  1


## Priprema podataka

Iz json datoteka potrebno je izvući ono bitno, a to su dvije liste potrebna za ubaciti u mrežu koju treniramo:
+ lista puteva do slika
+ lista koja označava kojoj kategoriji odgovara pojedina slika

In [ ]:
PATH_TO_JSON_FILE = "../fastener_dataset/annotations/instances_default.json"
PATH_TO_IMAGES_FOLDER = "../fastener_dataset/images"

In [ ]:
with open(PATH_TO_JSON_FILE, "r") as f:
    data = json.load(f)

images = data["images"]
print("Number of images: ",len(images))
print("Attributes: ",list(images[0].keys()))
print(images[-1])
print()

annotations = data["annotations"]
print("Number of annotations (they refer to images):",len(annotations))
print("Attributes: ", list(annotations[0].keys()))

print(annotations[-1])

Number of images:  16970
Attributes:  ['id', 'width', 'height', 'file_name', 'license', 'flickr_url', 'coco_url', 'date_captured']
{'id': 17000, 'width': 3456, 'height': 3456, 'file_name': '667/667-4/2022-09-19T11:32:12.200928+00:00_5ae41d00-775e-425e-b8cc-72ad3eb42455.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}

Number of annotations (they refer to images): 16970
Attributes:  ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes']
{'id': 17011, 'image_id': 17000, 'category_id': 6, 'segmentation': [], 'area': 4597566.259289555, 'bbox': [717.3, 708.9, 2149.4, 2139.0], 'iscrowd': 0, 'attributes': {'misc group': 'clamp', 'occluded': False, 'rotation': 0, 'size_code': '667-4', 'size_code_filesystem': '667-4', 'category': '667'}}


Bitno mi je samo images.file_name, annotations.category_id i annotations.category (za model s podklasama)

ID ne prati indeksni broj zapisa!

In [ ]:
paths = []
labels = []

for annotation in annotations:
    image_id = annotation["image_id"]

    file_name = ""
    #sram me ove petlje, vjerovatno se moze nekako aproksimirat uz pomoc image_id-a
    for image in images:
        if str(image["id"]) == str(image_id):
            file_name = image["file_name"]
            break

    #u jsonu se nalaze krivi nazivi slika pa je potrebno ispraviti
    paths.append(PATH_TO_IMAGES_FOLDER + "/" + file_name.replace(":","_"))
    labels.append(annotation["category_id"])

print("Size of paths and labels should be the same: ", len(paths), len(labels))

In [ ]:
with open("artifacts/paths_labels.json", "w") as f:
    json.dump(
        {"paths": paths, "labels": labels},
        f,
        indent=2
    )